In [1]:
import pandas as pd
import numpy as np
import epiweeks
import os

### Get Metadata and Labels

In [2]:
violence_sdoh = pd.read_csv('Metadata/Metadata.csv')
violence_sdoh

,Municipality code,Municipality,Population2007,Population2008,Population2009,Population2010,Population2011,Population2012,Population2013,Population2014,...,202322,202323,202324,202325,202326,202327,202328,202329,202330,202331
0,5001,Medellín,2286126,2309689,2331389,2353410,2379920,2410046,2428850,2436071,...,171,150,148,162,140,137,148,145,119,45
1,5002,Abejorral,20643,20944,21197,21355,20973,20067,19732,19796,...,2,1,0,0,1,0,0,4,0,0
2,5004,Abriaquí,2261,2290,2321,2353,2386,2424,2471,2488,...,0,0,0,0,0,0,0,0,0,0
3,5021,Alejandría,3935,4020,4096,4171,4244,4337,4419,4467,...,0,0,0,0,0,0,0,0,0,0
4,5030,Amagá,27996,28331,28624,28839,28776,28371,28259,28432,...,1,1,3,1,1,1,3,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096,97666,Taraira,432,540,689,891,1148,1355,1492,1612,...,0,0,1,0,0,0,0,1,0,0
1097,99001,Puerto Carreño,13742,14559,15346,16124,16934,17773,18635,19519,...,0,0,0,0,3,1,1,1,1,1
1098,99524,La Primavera,5190,5784,6365,6920,7496,8034,8617,9166,...,0,0,0,0,0,0,1,0,0,0
1099,99624,Santa Rosalía,3167,3378,3586,3846,4067,4325,4535,4635,...,0,0,0,0,0,0,0,0,0,0


In [3]:
violence_sdoh.to_csv('metadata_labels.csv', index=False)

### Get Internet data

#### Google Trends

In [4]:
def transform_dataframe(df, column_name):
    melted_df = pd.melt(df, id_vars=['epiweek'], var_name='Municipality Code', value_name=column_name)
    return melted_df[['Municipality Code', 'epiweek', column_name]]


In [5]:
def get_and_merge(path, dfs):
    
    for i, column_name in enumerate(dfs):
        if i == 0:
            df = pd.read_csv(path + column_name + '.csv')
            df.drop(columns=['date'], inplace=True)
            df = transform_dataframe(df, column_name)
        else:
            temp = pd.read_csv(path + column_name + '.csv')
            temp.drop(columns=['date'], inplace=True)
            temp = transform_dataframe(temp, column_name)
            df = df.merge(temp, on=['Municipality Code', 'epiweek'])
    return df


In [6]:
google_df = get_and_merge(path='Internet/Google/', dfs=['Violencia', 'violencia de genero'])
google_df

,Municipality Code,epiweek,Violencia,violencia de genero
0,Medellín,201752,16,0
1,Medellín,201801,20,32
2,Medellín,201802,14,0
3,Medellín,201803,31,22
4,Medellín,201804,31,2
...,...,...,...,...
2625,Cúcuta,202249,9,0
2626,Cúcuta,202250,28,0
2627,Cúcuta,202251,18,17
2628,Cúcuta,202252,11,30


In [7]:
google_df.to_csv('google_data.csv', index=False)

#### Media Cloud

In [8]:
media_cloud = pd.read_csv('Internet/Media Cloud/labels.csv')
media_cloud

,Municipality code,publish_date,title
0,Popayán,2018-01-03 15:51:29,Dos capturados en Cartagena por violencia intr...
1,Popayán,2018-01-04 08:24:23,Hallan cadáver de Yuly Tatiana Sánchez en caño...
2,Popayán,2018-01-04 11:47:04,Programa de Familias Fuertes cerró el 2017 con...
3,Popayán,2018-01-04 11:47:04,Programa de Familias Fuertes cerró el 2017 con...
4,Popayán,2018-01-05 07:10:27,Inspección de Ejército investiga incidente ent...
...,...,...,...
7045,Pasto,2023-11-18 18:55:06,Otro intento de violación en Kennedy
7046,Pasto,2023-11-18 22:04:00,Las declaraciones de Mancuso sobre Uribe en re...
7047,Pasto,2023-11-19 14:55:50,Garantía de seguridad mediante patrullajes
7048,Pasto,2023-11-19 16:09:03,Entre lágrimas celebró una leyenda del fútbol ...


In [9]:
media_cloud['publish_date'] = media_cloud['publish_date'].apply(lambda x: x[:10])
media_cloud['publish_date'] = pd.to_datetime(media_cloud['publish_date'])
media_cloud['epiweek'] = media_cloud['publish_date'].apply(lambda x: epiweeks.Week.fromdate(x))

# Group by 'Municipality code' and 'epiweek', join titles, and add a count column
media_cloud = media_cloud.groupby(['Municipality code', 'epiweek'])['title'].apply(lambda x: '; '.join(x)).reset_index()
media_cloud['Count'] = media_cloud.groupby(['Municipality code', 'epiweek'])['title'].transform('count')

media_cloud

,Municipality code,epiweek,title,Count
0,Barranquilla,201847,Procuraduría se opone a que Maza Márquez vaya ...,1
1,Barranquilla,201849,Una mujer fallece en ataque a migrantes centro...,1
2,Barranquilla,201852,Duque emplaza a ‘el Paisa’ para que comparezca...,1
3,Barranquilla,201903,Más de 11 mil personas se han sometido a la JE...,1
4,Barranquilla,201904,OEA no logra consenso ante crisis de Venezuela,1
...,...,...,...,...
966,Popayán,202343,La última guerra del mundo; Procuraduría alert...,1
967,Popayán,202344,"Carlos F Galán, el hijo del fallecido Luis Car...",1
968,Popayán,202345,EL EJÉRCITO DE ISRAEL HA MATADO YA A MÁS DE 90...,1
969,Popayán,202346,El ángel de los cuchillos: una macabra escultu...,1


#### GDELT

In [10]:
gdelt = pd.read_csv('Internet/gdelt/labels_violence_cohort.csv')
gdelt

,Municipality code,SQLDATE,Actor1Name,Actor2Name,NumMentions,NumSources,NumArticles,AvgTone,SOURCEURL,ExtractedText,ContentTitle
0,Villavicencio,20180903,FARC,NaN,5,1,5,-9.230769,https://morningstaronline.co.uk/article/colomb...,Reset on: COLOMBIA brought charges against b...,LATEST EDITORIAL
1,Villavicencio,20180903,CIVILIAN,NaN,3,1,3,-9.230769,https://morningstaronline.co.uk/article/colomb...,Reset on: COLOMBIA brought charges against b...,LATEST EDITORIAL
2,Villavicencio,20180903,NaN,FARC,5,1,5,-9.230769,https://morningstaronline.co.uk/article/colomb...,Reset on: COLOMBIA brought charges against b...,LATEST EDITORIAL
3,Villavicencio,20180402,COLOMBIA,NaN,8,1,8,-4.643963,https://www.telesurtv.net/english/news/One-Mor...,teleSURHD Opacity If there is not a Board of ...,Guatemala: Tensions and Delay in the Formation...
4,Villavicencio,20180403,RIGHTS GROUP,NaN,10,1,10,-4.713805,https://www.aljazeera.com/news/2018/03/rights-...,"The Colombian social leader, known as an ‘unti...",Skip links
...,...,...,...,...,...,...,...,...,...,...,...
10697,Cúcuta,20201118,COLOMBIA,NaN,12,5,12,-5.888438,https://www.pressherald.com/2020/11/17/hurrica...,\n\n\n\nSign in or Subscribe \nSee Offers\n\n\...,Sign In:
10698,Cúcuta,20201118,COLOMBIAN,NaN,25,6,25,-5.701540,https://www.pressherald.com/2020/11/17/hurrica...,\n\n\n\nSign in or Subscribe \nSee Offers\n\n\...,Sign In:
10699,Cúcuta,20201118,PRESIDENT,NaN,3,1,3,-4.580153,https://colombiareports.com/colombia-rescues-d...,Colombia’s air force rescued more than 50 peop...,Colombia rescues dozens from island destroyed ...
10700,Cúcuta,20210102,CARRIER,NaN,5,1,5,-1.003344,https://www.flightglobal.com/safety/avianca-a3...,NaN,NaN


In [11]:
# Convert SQLDATE to empiweek in the yearweek format
gdelt['epiweek'] = gdelt['SQLDATE'].apply(lambda x: epiweeks.Week.fromdate(pd.to_datetime(str(x), format='%Y%m%d')))
gdelt = gdelt[['Municipality code', 'epiweek', 'Actor1Name', 'NumMentions', 
               'NumSources', 'NumArticles', 'AvgTone', 'ExtractedText']]

# Group by epiweek and Municipality code and perform aggregations
gdelt_preproc = gdelt.groupby(['epiweek', 'Municipality code']).agg({
    'Actor1Name': lambda x: ', '.join(x.dropna()) if x.notna().any() else None,
    'NumMentions': 'sum',
    'NumSources': 'sum',
    'NumArticles': 'sum',
    'AvgTone': 'mean',
    'ExtractedText': lambda x: '; '.join(x.dropna()) if x.notna().any() else None,
}).reset_index()

# Create a counter for the number of posts per epiweek
gdelt_preproc['PostCount'] = gdelt.groupby(['epiweek', 'Municipality code']).size().reset_index(name='PostCount')['PostCount']
gdelt_preproc

,epiweek,Municipality code,Actor1Name,NumMentions,NumSources,NumArticles,AvgTone,ExtractedText,PostCount
0,201801,Cali,FARC,10,1,10,-4.921260,None,1
1,201801,Medellín,"COLOMBIA, COLOMBIA, GOVERNMENT, COLOMBIA, COLO...",82,15,66,-6.511003,An attack on a nightclub in an important cocai...,14
2,201801,Pasto,"POLICE, COLOMBIA, COLOMBIA",25,5,25,-3.281993,The ELN is the only guerrilla group still at w...,5
3,201801,Popayán,"GUNMEN, POLICE, COLOMBIA, COLOMBIA, GUNMEN, GU...",19,6,19,-5.982906,None,6
4,201801,Soacha,"COMMUNITY, BOGOTA, COLOMBIAN, COLOMBIAN, COLOM...",71,12,71,-9.385465,None,12
...,...,...,...,...,...,...,...,...,...
1347,202250,Soacha,BOGOTA,10,1,10,-4.054054,"CHARLOTTE, N.C. (QUEEN CITY NEWS) – The Foo Fi...",1
1348,202251,Barranquilla,"TANKER, BARRANQUILLA",20,2,20,-5.161290,"Barranquilla, Colombia, Dec 21 (EFE).- A firef...",2
1349,202251,Medellín,None,4,1,4,-4.334365,We have gotten word of a horrifying story unfo...,1
1350,202251,Pasto,"COLOMBIAN, COLOMBIAN",30,2,30,-9.833024,Police offices in Colombia are documenting “ex...,2


#### Merge gdelt and media cloud data

In [12]:
# Merge the two DataFrames on 'epiweek' and 'Municipality code'
merged_df = pd.merge(gdelt_preproc, media_cloud, on=['epiweek', 'Municipality code'], how='outer')

# Fill NaN values in Count with 0
merged_df['Count'].fillna(0, inplace=True)
merged_df['PostCount'].fillna(0, inplace=True)
merged_df['Count'] = merged_df['Count'] + merged_df['PostCount']
merged_df.drop(columns=['PostCount'], inplace=True)

# Concatenate 'title' and 'Actor1Name' columns
merged_df['title'] = merged_df['Actor1Name'].fillna('') + ';' + merged_df['title'].fillna('')
# Drop the 'Actor1Name' column
merged_df.drop(columns=['Actor1Name'], inplace=True)

# Concatenate 'title' and 'ExtractedText' columns
#merged_df['title'] = merged_df['title'].fillna('') + ';' + merged_df['ExtractedText'].fillna('')
# Drop the 'ExtractedText' column
merged_df.drop(columns=['ExtractedText'], inplace=True)

# Group by 'epiweek' and 'Municipality code' and perform aggregation
media_df = merged_df.groupby(['Municipality code', 'epiweek']).agg({
    'Count': 'sum',
    'title': lambda x: ';'.join(x),
    'NumMentions': 'sum',
    'NumSources': 'sum',
    'NumArticles': 'sum',
    'AvgTone': 'mean',
}).reset_index()

media_df

,Municipality code,epiweek,Count,title,NumMentions,NumSources,NumArticles,AvgTone
0,Barranquilla,201803,5.0,"POLICE, COLOMBIA, PROSECUTOR, COLOMBIA, PROSEC...",27.0,5.0,27.0,-7.142799
1,Barranquilla,201804,8.0,"BARRANQUILLA, COLOMBIA, BARRANQUILLA, NEWSPAPE...",127.0,15.0,127.0,-7.153317
2,Barranquilla,201805,51.0,"POLICE, POLICE OFFICER, POLICE OFFICER, ASSAIL...",2444.0,277.0,2166.0,-6.787119
3,Barranquilla,201806,2.0,"BARRANQUILLA, COLOMBIA;",15.0,2.0,15.0,-4.870645
4,Barranquilla,201807,6.0,"GUERRILLA, FARC, GOVERNMENT FORCES, COLOMBIAN,...",44.0,6.0,44.0,-4.532365
...,...,...,...,...,...,...,...,...
2153,Villavicencio,202128,4.0,"COLOMBIA, POLICE, POLICE, POLICE;",7.0,4.0,7.0,-5.732946
2154,Villavicencio,202131,1.0,;,6.0,1.0,6.0,-5.121294
2155,Villavicencio,202137,3.0,"NATIONAL POLICE, COLOMBIA, CRIMINAL;",18.0,3.0,18.0,-7.229890
2156,Villavicencio,202140,2.0,"COLOMBIA, CRIMINAL;",6.0,2.0,6.0,-3.164557


In [13]:
media_df.to_csv('media_data.csv', index=False)

### Merge Google and News data

In [6]:
news = pd.read_csv('media_data.csv')
news

,Municipality code,epiweek,Count,title,NumMentions,NumSources,NumArticles,AvgTone
0,Barranquilla,201803,5.0,"POLICE, COLOMBIA, PROSECUTOR, COLOMBIA, PROSEC...",27.0,5.0,27.0,-7.142799
1,Barranquilla,201804,8.0,"BARRANQUILLA, COLOMBIA, BARRANQUILLA, NEWSPAPE...",127.0,15.0,127.0,-7.153317
2,Barranquilla,201805,51.0,"POLICE, POLICE OFFICER, POLICE OFFICER, ASSAIL...",2444.0,277.0,2166.0,-6.787119
3,Barranquilla,201806,2.0,"BARRANQUILLA, COLOMBIA;",15.0,2.0,15.0,-4.870645
4,Barranquilla,201807,6.0,"GUERRILLA, FARC, GOVERNMENT FORCES, COLOMBIAN,...",44.0,6.0,44.0,-4.532365
...,...,...,...,...,...,...,...,...
2153,Villavicencio,202128,4.0,"COLOMBIA, POLICE, POLICE, POLICE;",7.0,4.0,7.0,-5.732946
2154,Villavicencio,202131,1.0,;,6.0,1.0,6.0,-5.121294
2155,Villavicencio,202137,3.0,"NATIONAL POLICE, COLOMBIA, CRIMINAL;",18.0,3.0,18.0,-7.229890
2156,Villavicencio,202140,2.0,"COLOMBIA, CRIMINAL;",6.0,2.0,6.0,-3.164557


In [7]:
google = pd.read_csv('google_data.csv')
google

,Municipality Code,epiweek,Violencia,violencia de genero
0,Medellín,201752,16,0
1,Medellín,201801,20,32
2,Medellín,201802,14,0
3,Medellín,201803,31,22
4,Medellín,201804,31,2
...,...,...,...,...
2625,Cúcuta,202249,9,0
2626,Cúcuta,202250,28,0
2627,Cúcuta,202251,18,17
2628,Cúcuta,202252,11,30


In [8]:
internet = pd.merge(google, news, right_on=['Municipality code', 'epiweek'], left_on=['Municipality Code', 'epiweek'], how='left')
internet.drop(columns=['title', 'Municipality Code'], inplace=True)
# drom nan in Municipality code
internet.dropna(subset=['Municipality code'], inplace=True)
internet.fillna(0, inplace=True)
internet

,epiweek,Violencia,violencia de genero,Municipality code,Count,NumMentions,NumSources,NumArticles,AvgTone
1,201801,20,32,Medellín,14.0,82.0,15.0,66.0,-6.511003
2,201802,14,0,Medellín,7.0,38.0,7.0,38.0,-1.518428
3,201803,31,22,Medellín,20.0,107.0,20.0,107.0,-3.412981
4,201804,31,2,Medellín,6.0,37.0,6.0,37.0,-4.515344
5,201805,38,0,Medellín,13.0,50.0,13.0,50.0,-5.697037
...,...,...,...,...,...,...,...,...,...
2618,202242,58,0,Cúcuta,2.0,20.0,2.0,20.0,-3.795380
2619,202243,67,1,Cúcuta,4.0,26.0,4.0,26.0,-4.112392
2620,202244,29,19,Cúcuta,2.0,10.0,2.0,10.0,-5.146430
2625,202249,9,0,Cúcuta,2.0,5.0,2.0,5.0,-6.451613


In [9]:
internet.to_csv('internet_data.csv', index=False)